# Morphologic Transformations Visualizations 🔲

This notebook contains algorithms for visualizing some basic morphologic transformations such as erosion, dilation, opening, and closing. 

It serves the purpose of offering a better understanding in the process of exploring Image Processing. 📚

In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Dilation and Erosion

In [2]:
input_img = cv.imread("./img/mon1thr1_bw.bmp", cv.IMREAD_GRAYSCALE)
# cv.imshow("input", input_img)
# cv.waitKey(0)                    # press on any key to close the image
# cv.destroyWindow("input") 

## Out-of-box behavior

In [3]:
cv.imshow("source", input_img)

default_dilated = cv.dilate(input_img, None, iterations = 1)

default_eroded = cv.erode(input_img, None, iterations = 1)

cv.imshow("default dilation", default_dilated)
cv.imshow("default eroded", default_eroded)

cv.waitKey(0)
cv.destroyWindow("source")
cv.destroyWindow("default dilation")
cv.destroyWindow("default eroded")

*Question: Why is it reversed?*

## Implementation and Visualization

In [4]:
def getOffsets(structuring_elem):
    elem_matrix, origin = structuring_elem
    x_origin, y_origin = origin

    (x_coord, y_coord) = np.where(elem_matrix == 1)    
    x_coord = x_coord - x_origin    
    y_coord = y_coord - y_origin
    
    offsets = list(zip(x_coord, y_coord))
    return offsets
    
def inImage(curr_i, curr_j, rows, cols):
    if curr_i > 0 and curr_i < rows:
        if curr_j > 0 and curr_j < cols:
            return True
    return False

def customErosion(src, structuring_elem, iterations, bg_col):
    """
    Performs erosion on src and returns the eroded image.
    Parameters:
    - src - 2D numpy array of uintc
    - structuring element - tuple (2D numpy array of uintc filled with 1s or 0s, origin_coordinates)
    """
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(dst)
    structuring_elem_offsets = getOffsets(structuring_elem)

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    all_neigbors_are_object = True
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            curr_neigh = src[i + offset_i, j + offset_j] if iter == 0 else dst[i + offset_i, j + offset_j]
                            if curr_neigh == 255:
                                all_neigbors_are_object = False
                                break
                    if all_neigbors_are_object:
                        aux[i, j] = 0 # color the origin as object
                    else:
                        aux[i, j] = 255
        dst = np.copy(aux)

    return dst
                            

def customDilation(src, structuring_elem, iterations, bg_col):
    """
    Performs dilation on src and returns the dilated image.
    Parameters:
    - src - 2D numpy array of uintc
    - structuring element - tuple (2D numpy array of uintc filled with 1s or 0s, origin_coordinates)
    """
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(dst)
    structuring_elem_offsets = getOffsets(structuring_elem)

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            aux[i + offset_i, j + offset_j] = 0
        dst = np.copy(aux)
    return dst


def main() :
    structuring_elem = (np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]), (1, 1))
    compute = False # Modify if needed

    if compute:
        eroded = customErosion(input_img, structuring_elem, 1, 1)
        dilated = customDilation(input_img, structuring_elem, 1, 1)
        cv.imshow("source", input_img)
        cv.imshow("eroded", eroded)
        cv.imshow("dilated", dilated)

        cv.waitKey(0)
        cv.destroyWindow("eroded")
        cv.destroyWindow("dilated")
        cv.destroyWindow("source")


if __name__ == '__main__':
    main()

### Creating animations to show the workings of erosion and dilation
Ideas:
- show intermmediary steps (export pictures and combine them in a small video)
    - try to zoom in somehow
- overlap the images in a color image

In [27]:
import os
import imageio # for creating the gif

def zoom_in(src, percent):
    height, width, _ = src.shape
    new_height = int(height * percent / 100) 
    new_width = int(width * percent / 100)
    dst = cv.resize(src, (new_width, new_height), interpolation = cv.INTER_NEAREST)
    return dst

def overlapStructElemAndSave(src, i, j, offsets):
    """
    Create an image that overlaps the structuring element and the source
    """
    rows, cols = src.shape
    to_return= np.full((rows, cols, 3), 255, np.uint8) # White bg

    # Leave object pixels black
    to_return[:, :, 0] = src
    to_return[:, :, 1] = src
    to_return[:, :, 2] = src

    for offset_i, offset_j in offsets:
        index_i = i + offset_i
        index_j = j + offset_j
        if(to_return[index_i, index_j, 1] == 0): # intersection between struct elem and object (dark green)
            to_return[i + offset_i, j + offset_j, 1] = 99
            to_return[i + offset_i, j + offset_j, 2] = 18
        else: # structuring element outside object (green)
            to_return[i + offset_i, j + offset_j, 0] = 0
            to_return[i + offset_i, j + offset_j, 2] = 0

    # highlight object center (yellow)
    to_return[i, j, 1] = 255
    to_return[i, j, 2] = 255

    return to_return

def customDilationWithVisualization(src, structuring_elem, iterations, bg_col):
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(src)
    structuring_elem_offsets = getOffsets(structuring_elem)

    ret_images = []

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    last_i, last_j = i,j
                    overlapped = overlapStructElemAndSave(aux, i, j, structuring_elem_offsets)
                    ret_images.append(zoom_in(overlapped, 4000))
                    
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            aux[i + offset_i, j + offset_j] = 0
        dst = np.copy(aux)
    
    return ret_images

def customErosionWithVisualization(src, structuring_elem, iterations, bg_col):
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(src)
    structuring_elem_offsets = getOffsets(structuring_elem)

    ret_images = []

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    last_i, last_j = i, j
                    
                    ret_images.append(overlapStructElemAndSave(aux, i, j, structuring_elem_offsets))

                    all_neigbors_are_object = True
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            curr_neigh = src[i + offset_i, j + offset_j] if iter == 0 else dst[i + offset_i, j + offset_j]
                            if curr_neigh == 255:
                                all_neigbors_are_object = False
                                break
                    if all_neigbors_are_object:
                        aux[i, j] = 0 # color the origin as object
                    else:
                        aux[i, j] = 255
        ret_images.append(overlapStructElemAndSave(aux, last_i, last_j, structuring_elem_offsets))
        dst = np.copy(aux)
    
    return ret_images


def videoFromImages(src_frames, out_path):
    imageio.mimsave(os.path.join(out_path), src_frames, duration=200)
    return

def main() :
    input_img = cv.imread("./img/lines.bmp", cv.IMREAD_GRAYSCALE)
    structuring_elem = (np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]), (1, 1))
    dilation_frames = customDilationWithVisualization(input_img, structuring_elem, 1, 1)

    videoFromImages(dilation_frames, "./outDilation/dilation_gif.gif")
    
    # input_erosion = cv.imread("./img/inf.bmp", cv.IMREAD_GRAYSCALE)
    # erosion_frames = customErosionWithVisualization(input_erosion, structuring_elem, 1, 1)
    # videoFromImages(erosion_frames, "./outErosion/erosion_gif.gif")

if __name__ == '__main__':
    main()

### Questions
- how to zoom in, should I scale the image s.t. it shows larger squares that represent pixels?
    - A: on the same image  have a scaled portion (maybe imzoom with a certain intepolation technique NEAREST_NEIGHBOR)
- what next?
    - opening - neh
    - closing - neh
    - boundary extraction? - nope
    - region filling? - of interest - some iterations in a small object
    - other ones? Convex Hull, Skeletons, Pruning (not really)